In [88]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [89]:
data = pd.read_csv('datasets/worldcup2023.csv')

In [90]:
matches = pd.read_csv('datasets/matches.csv')

In [91]:
data.columns

Index(['Unnamed: 0', 'match_id', 'season', 'start_date', 'venue', 'innings',
       'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes',
       'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed'],
      dtype='object')

In [92]:
matches.columns

Index(['season', 'team1', 'team2', 'date', 'match_number', 'venue', 'city',
       'toss_winner', 'toss_decision', 'player_of_match', 'umpire1', 'umpire2',
       'reserve_umpire', 'match_referee', 'winner', 'winner_runs',
       'winner_wickets', 'match_type'],
      dtype='object')

Performing Data Cleaning.

In [93]:
matches.isna().sum()

season              0
team1               0
team2               0
date                0
match_number        0
venue               0
city                0
toss_winner         9
toss_decision       9
player_of_match     9
umpire1            10
umpire2            10
reserve_umpire     11
match_referee      10
winner              0
winner_runs        21
winner_wickets     21
match_type          0
dtype: int64

In [94]:
data.isna().sum()

Unnamed: 0                    0
match_id                      0
season                        0
start_date                    0
venue                         0
innings                       0
ball                          0
batting_team                  0
bowling_team                  0
striker                       0
non_striker                   0
bowler                        0
runs_off_bat                  0
extras                        0
wides                     21999
noballs                   22529
byes                      22547
legbyes                   22414
penalty                   22578
wicket_type               21938
player_dismissed          21938
other_wicket_type         22578
other_player_dismissed    22578
dtype: int64

In [95]:
matches.winner_runs.fillna(0, inplace=True)
matches.winner_wickets.fillna(0, inplace=True)

In [96]:
#filling na values
data[["wides", "noballs", "byes", "legbyes", "penalty"]] = data[["wides", "noballs", "byes", "legbyes", "penalty"]].fillna(0)

In [97]:
data.wicket_type.fillna("no wicket", inplace=True)
data.player_dismissed.fillna("no wicket", inplace=True)

In [98]:
data[["other_wicket_type", "other_player_dismissed"]] = data[["other_wicket_type", "other_player_dismissed"]].fillna("no dismissal")

In [99]:
# Getting the match number from the Match ID.
data['match_number'] = data['match_id'].rank(method='dense').astype(int)

In [100]:
# Keeping only the useful columns of this dataset
matches = matches[['season', 'team1', 'team2', 'date', 'match_number', 'venue', 'city','winner', 'winner_runs', 'winner_wickets']]

In [101]:
#Calculate the total runs scored on each ball
data["total_runs"] = data["runs_off_bat"] + data["extras"]

In [102]:
#creating a wicket column to get boolean of wicket fall
data["wicket"] = data["player_dismissed"].apply(lambda x: 1 if isinstance(x, str) else 0)

In [103]:
#Creating two columns of Over and Ball numbers
data["ball"] = data["ball"].astype(str)
data[["over", "ball_num"]] = data["ball"].str.split(".", expand=True).astype(int)
data["ball"] = data["ball"].astype(np.float64)
data["over"] = data["over"] + 1
data["ball_left"] = 306 - (data["over"]*6 + data["ball_num"])

In [104]:
#creating total runs column to calculate runs on each ball incusive of extras
data["total_runs"] = data["runs_off_bat"] + data["extras"]

Trying to build up a model.

In [105]:
#creating innings df so we can merger total runs for each innings with in original df
innings = data.groupby(["match_id", "innings"])["total_runs"].sum().reset_index()
innings.rename(columns={"total_runs":"innings_total"}, inplace=True)

In [106]:
deliveries = data.merge(innings, on=["match_id", "innings"])
deliveries

,Unnamed: 0,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,player_dismissed,other_wicket_type,other_player_dismissed,match_number,total_runs,wicket,over,ball_num,ball_left,innings_total
0,128024,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,...,no wicket,no dismissal,no dismissal,1,0,1,1,1,299,282
1,128025,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,...,no wicket,no dismissal,no dismissal,1,6,1,1,2,298,282
2,128026,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,...,no wicket,no dismissal,no dismissal,1,1,1,1,3,297,282
3,128027,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,...,no wicket,no dismissal,no dismissal,1,1,1,1,4,296,282
4,128028,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,...,no wicket,no dismissal,no dismissal,1,4,1,1,5,295,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22574,150598,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.6,South Africa,Afghanistan,AL Phehlukwayo,...,no wicket,no dismissal,no dismissal,42,1,1,47,6,18,247
22575,150599,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.7,South Africa,Afghanistan,HE van der Dussen,...,no wicket,no dismissal,no dismissal,42,2,1,47,7,17,247
22576,150600,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.1,South Africa,Afghanistan,AL Phehlukwayo,...,no wicket,no dismissal,no dismissal,42,6,1,48,1,17,247
22577,150601,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.2,South Africa,Afghanistan,AL Phehlukwayo,...,no wicket,no dismissal,no dismissal,42,4,1,48,2,16,247


In [107]:
deliveries["cumulative_runs"] = deliveries.groupby(["match_id", "innings"])["total_runs"].cumsum()
deliveries

,Unnamed: 0,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,other_wicket_type,other_player_dismissed,match_number,total_runs,wicket,over,ball_num,ball_left,innings_total,cumulative_runs
0,128024,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,...,no dismissal,no dismissal,1,0,1,1,1,299,282,0
1,128025,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,...,no dismissal,no dismissal,1,6,1,1,2,298,282,6
2,128026,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,...,no dismissal,no dismissal,1,1,1,1,3,297,282,7
3,128027,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,...,no dismissal,no dismissal,1,1,1,1,4,296,282,8
4,128028,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,...,no dismissal,no dismissal,1,4,1,1,5,295,282,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22574,150598,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.6,South Africa,Afghanistan,AL Phehlukwayo,...,no dismissal,no dismissal,42,1,1,47,6,18,247,229
22575,150599,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.7,South Africa,Afghanistan,HE van der Dussen,...,no dismissal,no dismissal,42,2,1,47,7,17,247,231
22576,150600,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.1,South Africa,Afghanistan,AL Phehlukwayo,...,no dismissal,no dismissal,42,6,1,48,1,17,247,237
22577,150601,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.2,South Africa,Afghanistan,AL Phehlukwayo,...,no dismissal,no dismissal,42,4,1,48,2,16,247,241


In [108]:
#creating run rate column
deliveries["run_rate"] = deliveries.groupby(["match_id","innings"]).apply(lambda x: (x["cumulative_runs"]*6)/(300-x["ball_left"])).reset_index(level=[0,1], drop=True)

In [109]:
deliveries

,Unnamed: 0,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,other_player_dismissed,match_number,total_runs,wicket,over,ball_num,ball_left,innings_total,cumulative_runs,run_rate
0,128024,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,...,no dismissal,1,0,1,1,1,299,282,0,0.000000
1,128025,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,...,no dismissal,1,6,1,1,2,298,282,6,18.000000
2,128026,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,...,no dismissal,1,1,1,1,3,297,282,7,14.000000
3,128027,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,...,no dismissal,1,1,1,1,4,296,282,8,12.000000
4,128028,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,...,no dismissal,1,4,1,1,5,295,282,12,14.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22574,150598,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.6,South Africa,Afghanistan,AL Phehlukwayo,...,no dismissal,42,1,1,47,6,18,247,229,4.872340
22575,150599,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.7,South Africa,Afghanistan,HE van der Dussen,...,no dismissal,42,2,1,47,7,17,247,231,4.897527
22576,150600,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.1,South Africa,Afghanistan,AL Phehlukwayo,...,no dismissal,42,6,1,48,1,17,247,237,5.024735
22577,150601,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.2,South Africa,Afghanistan,AL Phehlukwayo,...,no dismissal,42,4,1,48,2,16,247,241,5.091549


In [110]:
# Defining a colum of Target given by the Team Batting in Innings 1
innings_1 = deliveries[deliveries["innings"] == 1]

innings_1 = innings_1.groupby("match_id")["cumulative_runs"].max().reset_index()
innings_1.rename(columns={"cumulative_runs": "target"}, inplace=True)

innings_2 = deliveries[deliveries["innings"] == 2]
innings_2 = innings_2.merge(innings_1, on="match_id", how="left")
innings_2["target"] = innings_2["target"]+1

In [111]:
#Making columns for Runs Needed and the Required Run Rate to achieve that target.
innings_2["runs_needed"] = innings_2["target"] - innings_2["cumulative_runs"]
innings_2["required_run_rate"] = (innings_2["runs_needed"]*6) / innings_2["ball_left"]

In [112]:
#Adding wickets remaining for Innings 1
innings_1_f = deliveries[deliveries["innings"] == 1]
# innings_1_f["wickets_remaining"] = innings_1_f.groupby("match_id")["wicket"].apply(lambda x: 10 - x.cumsum()).reset_index(level=0, drop=True)

In [113]:
#concatinating innings1,2
innings = pd.concat([innings_1_f,innings_2])

In [114]:
innings

,Unnamed: 0,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,...,wicket,over,ball_num,ball_left,innings_total,cumulative_runs,run_rate,target,runs_needed,required_run_rate
0,128024,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,...,1,1,1,299,282,0,0.000000,NaN,NaN,NaN
1,128025,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,...,1,1,2,298,282,6,18.000000,NaN,NaN,NaN
2,128026,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,...,1,1,3,297,282,7,14.000000,NaN,NaN,NaN
3,128027,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,...,1,1,4,296,282,8,12.000000,NaN,NaN,NaN
4,128028,1384392,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,...,1,1,5,295,282,12,14.400000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10025,150598,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.6,South Africa,Afghanistan,AL Phehlukwayo,...,1,47,6,18,247,229,4.872340,245.0,16.0,5.333333
10026,150599,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,46.7,South Africa,Afghanistan,HE van der Dussen,...,1,47,7,17,247,231,4.897527,245.0,14.0,4.941176
10027,150600,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.1,South Africa,Afghanistan,AL Phehlukwayo,...,1,48,1,17,247,237,5.024735,245.0,8.0,2.823529
10028,150601,1384433,2023/24,2023-11-10,"Narendra Modi Stadium, Ahmedabad",2,47.2,South Africa,Afghanistan,AL Phehlukwayo,...,1,48,2,16,247,241,5.091549,245.0,4.0,1.500000


In [115]:
innings.isna().sum()

Unnamed: 0                    0
match_id                      0
season                        0
start_date                    0
venue                         0
innings                       0
ball                          0
batting_team                  0
bowling_team                  0
striker                       0
non_striker                   0
bowler                        0
runs_off_bat                  0
extras                        0
wides                         0
noballs                       0
byes                          0
legbyes                       0
penalty                       0
wicket_type                   0
player_dismissed              0
other_wicket_type             0
other_player_dismissed        0
match_number                  0
total_runs                    0
wicket                        0
over                          0
ball_num                      0
ball_left                     0
innings_total                 0
cumulative_runs               0
run_rate

In [116]:
matches.columns

Index(['season', 'team1', 'team2', 'date', 'match_number', 'venue', 'city',
       'winner', 'winner_runs', 'winner_wickets'],
      dtype='object')

In [117]:
#merging with matches data to get winners and other data that can help our model
innings = innings.merge(matches[["match_number","winner", "winner_runs", "winner_wickets"]], on="match_number")

In [118]:
innings["result"] = innings.apply(lambda x: 1 if x["batting_team"] == x["winner"] else 0, axis=1)

In [119]:
innings.isna().sum()

Unnamed: 0                    0
match_id                      0
season                        0
start_date                    0
venue                         0
innings                       0
ball                          0
batting_team                  0
bowling_team                  0
striker                       0
non_striker                   0
bowler                        0
runs_off_bat                  0
extras                        0
wides                         0
noballs                       0
byes                          0
legbyes                       0
penalty                       0
wicket_type                   0
player_dismissed              0
other_wicket_type             0
other_player_dismissed        0
match_number                  0
total_runs                    0
wicket                        0
over                          0
ball_num                      0
ball_left                     0
innings_total                 0
cumulative_runs               0
run_rate

In [120]:
innings.fillna(0, inplace=True)

In [121]:
innings = innings.sample(frac=1)

In [122]:
innings.to_csv('datasets/innings.csv')

Building the Model

In [123]:
#Defining model data for prediction
model_cols = innings[["venue", "batting_team", "bowling_team", "ball", "innings_total", "run_rate", "required_run_rate",
                      "ball_left", "runs_needed", "target", "result"]]

In [124]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


# Extract features and target variable
X = model_cols.drop("result", axis=1)  # Features
y = model_cols["result"]  # Target variable

In [125]:
X.dtypes

venue                 object
batting_team          object
bowling_team          object
ball                 float64
innings_total          int64
run_rate             float64
required_run_rate    float64
ball_left              int32
runs_needed          float64
target               float64
dtype: object

In [126]:
categorical_cols = ["venue", "batting_team", "bowling_team"]
numeric_cols = list(set(X.columns) - set(categorical_cols))


# Create transformers for preprocessing
numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ("label_encoder", OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

In [127]:
numeric_cols

['required_run_rate',
 'runs_needed',
 'run_rate',
 'target',
 'innings_total',
 'ball',
 'ball_left']

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [129]:
print(X_train.shape[1])

10


In [130]:
X_train['required_run_rate'] = X_train['required_run_rate'].replace(np.inf, -1)

In [131]:
X_train.dtypes

venue                 object
batting_team          object
bowling_team          object
ball                 float64
innings_total          int64
run_rate             float64
required_run_rate    float64
ball_left              int32
runs_needed          float64
target               float64
dtype: object

In [132]:
X_train.shape

(18063, 10)

In [133]:
# Preprocess the training and testing data
X_train_preprocessed = preprocessor.fit_transform(X_train)
# X_test_preprocessed = preprocessor.transform(X_test)
X_train_preprocessed.shape

(18063, 37)

In [134]:
print(X_test[X_test.isin([np.nan, np.inf, -np.inf])].sum())

venue                  0
batting_team           0
bowling_team           0
ball                 0.0
innings_total        0.0
run_rate             0.0
required_run_rate    inf
ball_left            0.0
runs_needed          0.0
target               0.0
dtype: object


In [135]:
X_train.columns

Index(['venue', 'batting_team', 'bowling_team', 'ball', 'innings_total',
       'run_rate', 'required_run_rate', 'ball_left', 'runs_needed', 'target'],
      dtype='object')

In [136]:
X_test['required_run_rate'] = X_test['required_run_rate'].replace(np.inf, -1)

In [143]:
# Define the deep learning model
model = Sequential()
model.add(Dense(32, input_dim=X_train_preprocessed.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout with a rate of 0.5
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert the sparse matrix to a dense matrix
X_train_preprocessed_dense = X_train_preprocessed.toarray()

# Fit the model
model.fit(X_train_preprocessed_dense, y_train, epochs=10, batch_size=8, validation_split=0.2)


Epoch 1/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.3098 - accuracy: 0.8941 - val_loss: 0.0383 - val_accuracy: 0.9989
Epoch 2/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0824 - accuracy: 0.9829 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 3/10
1807/1807 [==============================] - 3s 1ms/step - loss: 0.0567 - accuracy: 0.9837 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 4/10
1807/1807 [==============================] - 3s 1ms/step - loss: 0.0463 - accuracy: 0.9862 - val_loss: 3.3262e-04 - val_accuracy: 1.0000
Epoch 5/10
1807/1807 [==============================] - 3s 1ms/step - loss: 0.0421 - accuracy: 0.9880 - val_loss: 1.8962e-04 - val_accuracy: 1.0000
Epoch 6/10
1807/1807 [==============================] - 3s 1ms/step - loss: 0.0413 - accuracy: 0.9878 - val_loss: 8.8904e-05 - val_accuracy: 1.0000
Epoch 7/10
1807/1807 [==============================] - 3s 2ms/step - loss: 0.0314 - accuracy: 0.9922 - val_loss: 7.4045e-05

In [144]:
X_test[:10].values

array([['M Chinnaswamy Stadium, Bengaluru', 'England', 'Sri Lanka', 32.3,
        156, 4.6461538461538465, 0.0, 105, 0.0, 0.0],
       ['MA Chidambaram Stadium, Chepauk, Chennai', 'Bangladesh',
        'New Zealand', 0.1, 245, 0.0, 0.0, 299, 0.0, 0.0],
       ['M Chinnaswamy Stadium, Bengaluru', 'Sri Lanka', 'New Zealand',
        14.5, 171, 6.337078651685394, 0.0, 211, 0.0, 0.0],
       ['Maharashtra Cricket Association Stadium, Pune', 'Afghanistan',
        'Sri Lanka', 28.3, 242, 4.807017543859649, 4.883720930232558,
        129, 105.0, 242.0],
       ['M Chinnaswamy Stadium, Bengaluru', 'Sri Lanka', 'New Zealand',
        32.4, 171, 3.979591836734694, 0.0, 104, 0.0, 0.0],
       ['Narendra Modi Stadium, Ahmedabad', 'New Zealand', 'England',
        11.2, 283, 8.205882352941176, 4.913793103448276, 232, 190.0,
        283.0],
       ['Rajiv Gandhi International Stadium, Uppal, Hyderabad',
        'Sri Lanka', 'Pakistan', 32.2, 344, 7.237113402061856, 0.0, 106,
        0.0, 0.0],
    

In [147]:
# Evaluate the model
accuracy = model.evaluate(preprocessor.transform(X_test).toarray(), y_test)[1]
print(f"Accuracy: {accuracy:.2f}")

142/142 [==============================] - 0s 1ms/step - loss: 1.0012e-05 - accuracy: 1.0000
Accuracy: 1.00


For Deployment Purpose, training with the complete dataset

In [148]:
# Extract features and target variable
X = model_cols.drop("result", axis=1)  # Features
y = model_cols["result"]  # Target variable

In [149]:
X['required_run_rate'] = X['required_run_rate'].replace(np.inf, -1)

In [150]:
X[:5]

,venue,batting_team,bowling_team,ball,innings_total,run_rate,required_run_rate,ball_left,runs_needed,target
1226,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,23.4,156,4.690141,0.0,158,0.0,0.0
7892,"MA Chidambaram Stadium, Chepauk, Chennai",New Zealand,Afghanistan,49.2,288,5.675676,0.0,4,0.0,0.0
13207,"M Chinnaswamy Stadium, Bengaluru",England,Sri Lanka,9.5,156,5.796610,0.0,241,0.0,0.0
17393,"Wankhede Stadium, Mumbai",India,Sri Lanka,2.2,357,5.142857,0.0,286,0.0,0.0
15417,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,India,England,22.3,229,3.600000,0.0,165,0.0,0.0


In [151]:
X_preprocessed = preprocessor.fit_transform(X)

In [152]:
#check if X_preprocessed is a pandas dataframe or numpy array
type(X_preprocessed)

scipy.sparse._csr.csr_matrix

In [153]:
X_preprocessed.shape

(22579, 37)

In [157]:
# Define the deep learning model
model = Sequential()
model.add(Dense(32, input_dim=X_preprocessed.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout with a rate of 0.5
model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X_preprocessed = X_preprocessed.toarray()

# Fit the model
model.fit(X_preprocessed, y, epochs=10, batch_size=1, validation_split=0.2)

Epoch 1/10
18063/18063 [==============================] - 25s 1ms/step - loss: 0.1411 - accuracy: 0.9287 - val_loss: 7.6821e-05 - val_accuracy: 1.0000
Epoch 2/10
18063/18063 [==============================] - 24s 1ms/step - loss: 0.0465 - accuracy: 0.9822 - val_loss: 2.0946e-07 - val_accuracy: 1.0000
Epoch 3/10
18063/18063 [==============================] - 25s 1ms/step - loss: 0.0462 - accuracy: 0.9833 - val_loss: 1.0714e-06 - val_accuracy: 1.0000
Epoch 4/10
18063/18063 [==============================] - 25s 1ms/step - loss: 0.0384 - accuracy: 0.9848 - val_loss: 1.7239e-09 - val_accuracy: 1.0000
Epoch 5/10
18063/18063 [==============================] - 24s 1ms/step - loss: 0.0460 - accuracy: 0.9822 - val_loss: 6.6680e-11 - val_accuracy: 1.0000
Epoch 6/10
18063/18063 [==============================] - 25s 1ms/step - loss: 0.0406 - accuracy: 0.9843 - val_loss: 4.1547e-12 - val_accuracy: 1.0000
Epoch 7/10
18063/18063 [==============================] - 24s 1ms/step - loss: 0.0416 - accura

In [158]:
#save the model
model.save('api/model.h5')

In [159]:
import joblib
joblib.dump(preprocessor, 'api/preprocessor.pkl')

['api/preprocessor.pkl']